In [1]:
import numpy as np
import pandas as pd

## Label Incoding EZone feature

In [2]:
from sklearn.preprocessing import LabelEncoder

## ARIMA on a single Bus stop per day

In [3]:
regular_travel = pd.read_csv('/kaggle/input/db-regio/regular_travel.csv')


In [4]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
labelencoder = labelencoder.fit(regular_travel['EZone'])
regular_travel['EZone'] = labelencoder.transform(regular_travel['EZone'])

In [5]:
reg_trav_bus_stop0=regular_travel[regular_travel['EZone']==0]
reg_trav_bus_stop0

,date,EZone,hour,Passengers
0,2019-01-01,0,0,0.0
1,2019-01-01,0,1,0.0
2,2019-01-01,0,2,0.0
3,2019-01-01,0,3,0.0
4,2019-01-01,0,4,0.0
...,...,...,...,...
418819,2019-12-16,0,19,NaN
418820,2019-12-16,0,20,NaN
418821,2019-12-16,0,21,NaN
418822,2019-12-16,0,22,NaN


In [6]:
reg_trav_bus_stop0.reset_index(inplace=True,drop=True)
reg_trav_bus_stop0

,date,EZone,hour,Passengers
0,2019-01-01,0,0,0.0
1,2019-01-01,0,1,0.0
2,2019-01-01,0,2,0.0
3,2019-01-01,0,3,0.0
4,2019-01-01,0,4,0.0
...,...,...,...,...
8395,2019-12-16,0,19,NaN
8396,2019-12-16,0,20,NaN
8397,2019-12-16,0,21,NaN
8398,2019-12-16,0,22,NaN


In [7]:
#create a copy of the original dataset
df_train=reg_trav_bus_stop0.copy()

Verify train and test indices

In [8]:
df_train.isna().sum()

date             0
EZone            0
hour             0
Passengers    2184
dtype: int64

In [9]:
#concat hour and date columns
df_train['date'] = pd.to_datetime(df_train['date'] + df_train['hour'].apply(lambda hour: str(hour).zfill(2)), format='%Y-%m-%d%H')
df_train 

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
8395,2019-12-16 19:00:00,0,19,NaN
8396,2019-12-16 20:00:00,0,20,NaN
8397,2019-12-16 21:00:00,0,21,NaN
8398,2019-12-16 22:00:00,0,22,NaN


In [10]:
df_train.dropna(subset=['Passengers'],inplace=True)
df_train

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
8227,2019-12-09 19:00:00,0,19,0.0
8228,2019-12-09 20:00:00,0,20,0.0
8229,2019-12-09 21:00:00,0,21,0.0
8230,2019-12-09 22:00:00,0,22,0.0


In [11]:
#Same work with test data
regular_travel_test = pd.read_csv('/kaggle/input/db-regio/regular_travel_test.csv')
regular_travel_test['EZone'] = labelencoder.transform(regular_travel_test['EZone'])
df_test=regular_travel_test[regular_travel_test['EZone']==0]
df_test

,date,EZone,hour,Passengers
0,2019-01-22,0,0,0
1,2019-01-22,0,1,0
2,2019-01-22,0,2,0
3,2019-01-22,0,3,0
4,2019-01-22,0,4,0
...,...,...,...,...
108019,2019-12-16,0,19,0
108020,2019-12-16,0,20,0
108021,2019-12-16,0,21,0
108022,2019-12-16,0,22,0


In [12]:
df_test['date'] = pd.to_datetime(df_test['date'] + df_test['hour'].apply(lambda hour: str(hour).zfill(2)), format='%Y-%m-%d%H')
df_test

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,EZone,hour,Passengers
0,2019-01-22 00:00:00,0,0,0
1,2019-01-22 01:00:00,0,1,0
2,2019-01-22 02:00:00,0,2,0
3,2019-01-22 03:00:00,0,3,0
4,2019-01-22 04:00:00,0,4,0
...,...,...,...,...
108019,2019-12-16 19:00:00,0,19,0
108020,2019-12-16 20:00:00,0,20,0
108021,2019-12-16 21:00:00,0,21,0
108022,2019-12-16 22:00:00,0,22,0


**Concat of test and train data**

In [13]:
df_busstop0=pd.concat([df_train,df_test])
df_busstop0

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
108019,2019-12-16 19:00:00,0,19,0.0
108020,2019-12-16 20:00:00,0,20,0.0
108021,2019-12-16 21:00:00,0,21,0.0
108022,2019-12-16 22:00:00,0,22,0.0


In [14]:
df_busstop0.sort_values(by='date',ascending=True,inplace=True)
df_busstop0

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
108019,2019-12-16 19:00:00,0,19,0.0
108020,2019-12-16 20:00:00,0,20,0.0
108021,2019-12-16 21:00:00,0,21,0.0
108022,2019-12-16 22:00:00,0,22,0.0


In [15]:
df_busstop0.reset_index(inplace=True,drop=True)
df_busstop0

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
8395,2019-12-16 19:00:00,0,19,0.0
8396,2019-12-16 20:00:00,0,20,0.0
8397,2019-12-16 21:00:00,0,21,0.0
8398,2019-12-16 22:00:00,0,22,0.0


 # Upsampling the minority class

In [16]:
#we will work only on one month to see the predictions
second_month_train=df_busstop0[672:1176]
#take the majority class 0 and 1
majority_class=second_month_train[second_month_train['Passengers']<2]
len(majority_class)

484

In [17]:
minority_class = second_month_train[second_month_train['Passengers'] >1]
minority_class_upsampled = minority_class.sample(replace=True, n=len(majority_class))

In [18]:
minority_class_upsampled

,date,EZone,hour,Passengers
1140,2019-02-17 12:00:00,0,12,2.0
707,2019-01-30 11:00:00,0,11,5.0
1168,2019-02-18 16:00:00,0,16,2.0
1045,2019-02-13 13:00:00,0,13,3.0
1087,2019-02-15 07:00:00,0,7,3.0
...,...,...,...,...
1087,2019-02-15 07:00:00,0,7,3.0
1167,2019-02-18 15:00:00,0,15,2.0
877,2019-02-06 13:00:00,0,13,2.0
1087,2019-02-15 07:00:00,0,7,3.0


In [19]:
#Add the upsampled minority class to the original second month data
data_upsampled = pd.concat([second_month_train, minority_class_upsampled])
data_upsampled

,date,EZone,hour,Passengers
672,2019-01-29 00:00:00,0,0,0.0
673,2019-01-29 01:00:00,0,1,0.0
674,2019-01-29 02:00:00,0,2,0.0
675,2019-01-29 03:00:00,0,3,0.0
676,2019-01-29 04:00:00,0,4,0.0
...,...,...,...,...
1087,2019-02-15 07:00:00,0,7,3.0
1167,2019-02-18 15:00:00,0,15,2.0
877,2019-02-06 13:00:00,0,13,2.0
1087,2019-02-15 07:00:00,0,7,3.0


In [20]:
#sort by date
data_upsampled.sort_values(by='date',inplace=True)
data_upsampled

,date,EZone,hour,Passengers
672,2019-01-29 00:00:00,0,0,0.0
673,2019-01-29 01:00:00,0,1,0.0
674,2019-01-29 02:00:00,0,2,0.0
675,2019-01-29 03:00:00,0,3,0.0
676,2019-01-29 04:00:00,0,4,0.0
...,...,...,...,...
1171,2019-02-18 19:00:00,0,19,0.0
1172,2019-02-18 20:00:00,0,20,0.0
1173,2019-02-18 21:00:00,0,21,0.0
1174,2019-02-18 22:00:00,0,22,0.0


# Creating some lags informations mean and std

In [21]:
data_upsampled['Passengers_lag_mean_24']=data_upsampled['Passengers'].rolling(window=24).mean()
data_upsampled['Passengers_lag_mean_48']=data_upsampled['Passengers'].rolling(window=48).mean()
data_upsampled

,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48
672,2019-01-29 00:00:00,0,0,0.0,NaN,NaN
673,2019-01-29 01:00:00,0,1,0.0,NaN,NaN
674,2019-01-29 02:00:00,0,2,0.0,NaN,NaN
675,2019-01-29 03:00:00,0,3,0.0,NaN,NaN
676,2019-01-29 04:00:00,0,4,0.0,NaN,NaN
...,...,...,...,...,...,...
1171,2019-02-18 19:00:00,0,19,0.0,1.791667,1.895833
1172,2019-02-18 20:00:00,0,20,0.0,1.708333,1.854167
1173,2019-02-18 21:00:00,0,21,0.0,1.625000,1.812500
1174,2019-02-18 22:00:00,0,22,0.0,1.541667,1.770833


In [23]:
data_upsampled['Passengers_lag_std_24']=data_upsampled['Passengers'].rolling(window=24).std()
data_upsampled['Passengers_lag_std_48']=data_upsampled['Passengers'].rolling(window=48).std()
data_upsampled.head()

,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48,Passengers_lag_std_24,Passengers_lag_std_48
672,2019-01-29 00:00:00,0,0,0.0,NaN,NaN,NaN,NaN
673,2019-01-29 01:00:00,0,1,0.0,NaN,NaN,NaN,NaN
674,2019-01-29 02:00:00,0,2,0.0,NaN,NaN,NaN,NaN
675,2019-01-29 03:00:00,0,3,0.0,NaN,NaN,NaN,NaN
676,2019-01-29 04:00:00,0,4,0.0,NaN,NaN,NaN,NaN


In [24]:
data_upsampled.isna().sum()

date                       0
EZone                      0
hour                       0
Passengers                 0
Passengers_lag_mean_24    23
Passengers_lag_mean_48    47
Passengers_lag_std_24     23
Passengers_lag_std_48     47
dtype: int64

In [25]:
#Input missing values
data_upsampled.fillna(value=0,inplace=True)
data_upsampled.isna().sum()

date                      0
EZone                     0
hour                      0
Passengers                0
Passengers_lag_mean_24    0
Passengers_lag_mean_48    0
Passengers_lag_std_24     0
Passengers_lag_std_48     0
dtype: int64

In [26]:
data_upsampled.shape

(988, 8)

**Same work with the test data**

In [27]:
data_second_test=df_busstop0[1176:1344]
data_second_test

,date,EZone,hour,Passengers
1176,2019-02-19 00:00:00,0,0,0.0
1177,2019-02-19 01:00:00,0,1,0.0
1178,2019-02-19 02:00:00,0,2,0.0
1179,2019-02-19 03:00:00,0,3,0.0
1180,2019-02-19 04:00:00,0,4,0.0
...,...,...,...,...
1339,2019-02-25 19:00:00,0,19,0.0
1340,2019-02-25 20:00:00,0,20,0.0
1341,2019-02-25 21:00:00,0,21,0.0
1342,2019-02-25 22:00:00,0,22,0.0


In [28]:
data_second_test['Passengers_lag_mean_24']=data_second_test['Passengers'].rolling(window=24).mean()
data_second_test['Passengers_lag_mean_48']=data_second_test['Passengers'].rolling(window=48).mean()
data_second_test

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48
1176,2019-02-19 00:00:00,0,0,0.0,NaN,NaN
1177,2019-02-19 01:00:00,0,1,0.0,NaN,NaN
1178,2019-02-19 02:00:00,0,2,0.0,NaN,NaN
1179,2019-02-19 03:00:00,0,3,0.0,NaN,NaN
1180,2019-02-19 04:00:00,0,4,0.0,NaN,NaN
...,...,...,...,...,...,...
1339,2019-02-25 19:00:00,0,19,0.0,0.458333,0.25
1340,2019-02-25 20:00:00,0,20,0.0,0.458333,0.25
1341,2019-02-25 21:00:00,0,21,0.0,0.458333,0.25
1342,2019-02-25 22:00:00,0,22,0.0,0.458333,0.25


In [32]:
data_second_test['Passengers_lag_std_24']=data_second_test['Passengers'].rolling(window=24).std()
data_second_test['Passengers_lag_std_48']=data_second_test['Passengers'].rolling(window=48).std()
data_second_test.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48,Passengers_lag_std_24,Passengers_lag_std_48
1176,2019-02-19 00:00:00,0,0,0.0,0.0,0.0,NaN,NaN
1177,2019-02-19 01:00:00,0,1,0.0,0.0,0.0,NaN,NaN
1178,2019-02-19 02:00:00,0,2,0.0,0.0,0.0,NaN,NaN
1179,2019-02-19 03:00:00,0,3,0.0,0.0,0.0,NaN,NaN
1180,2019-02-19 04:00:00,0,4,0.0,0.0,0.0,NaN,NaN


In [30]:
data_second_test.fillna(value=0,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


# Training and test

In [33]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:0000:010:01


In [34]:
from pmdarima import auto_arima

In [35]:
#verify the stationarity of the data
from statsmodels.tsa.stattools import adfuller
result=adfuller(data_upsampled['Passengers'])

In [36]:
result[1]

1.2275871570311655e-06

So we can ignore the null hypothesis(data is not stationary) and consider the data as stationary

**Model using the hour feature**

In [38]:
model=auto_arima(y=data_upsampled['Passengers'],X=data_upsampled[['hour']],trace=True,test='adf',stepwise=True,stationary=True,scoring='mse',seasonal=False,start_p=1, start_q=1)

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=1681.178, Time=0.45 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=3546.114, Time=0.22 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=1691.279, Time=0.24 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=2719.862, Time=0.42 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3717.942, Time=0.17 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=1682.493, Time=0.60 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=1682.608, Time=1.34 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=2314.077, Time=1.29 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=1681.916, Time=0.42 sec
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=1684.493, Time=3.23 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1693.607, Time=0.36 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept
Total fit time: 8.762 seconds


**Use only the number of passengers**

In [39]:
estimator=auto_arima(y=data_upsampled['Passengers'],trace=True,test='adf',stepwise=True,seasonal=False,scoring='mse',start_p=1, start_q=1)

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1691.722, Time=0.17 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=4206.974, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=1706.733, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=3241.204, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=1691.729, Time=0.19 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=1692.164, Time=0.41 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=2710.954, Time=0.22 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=1693.270, Time=0.18 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=1693.729, Time=0.74 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=1679.179, Time=0.25 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=2719.022, Time=0.17 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=1689.280, Time=0.08 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=1680.494, Time=0.35 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=1680.608, Time=0.93 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept

**Prediction with 'hour' feature**

In [45]:
prd=model.predict(X=data_second_test[['hour']],n_periods=168)
prd

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


988     0.098868
989     0.176458
990     0.249331
991     0.317771
992     0.382049
          ...   
1151    1.377030
1152    1.376878
1153    1.376726
1154    1.376574
1155    1.376422
Length: 168, dtype: float64

In [46]:
np.max(prd.to_numpy())

1.3798574308755491

In [47]:
#MSE
from sklearn.metrics import mean_squared_error
mean_squared_error(data_second_test['Passengers'],prd)

1.4919397377188346

**Prediction without 'hour' feature**

In [48]:
prd1=estimator.predict(n_periods=168)
prd1

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


988     0.095442
989     0.173274
990     0.246382
991     0.315055
992     0.379562
          ...   
1151    1.378152
1152    1.378155
1153    1.378158
1154    1.378160
1155    1.378163
Length: 168, dtype: float64

In [49]:
mean_squared_error(data_second_test['Passengers'],prd1)

1.4919844379431393